In [6]:
def avgAge(t):
    allIn=t[t.toLeague.isin(leagueList)].groupby('toCountry')['ageFloat'].mean().sort_values()
    allOut=t[t.fromLeague.isin(leagueList)].groupby('fromCountry')['ageFloat'].mean().sort_values()
    freeIn=t[t.toLeague.isin(leagueList)&(t.feeValue==0)].groupby('toCountry')['ageFloat'].mean().sort_values()
    freeOut=t[t.fromLeague.isin(leagueList)&(t.feeValue==0)].groupby('fromCountry')['ageFloat'].mean().sort_values()
    paidIn=t[t.toLeague.isin(leagueList)&(t.feeValue>0)].groupby('toCountry')['ageFloat'].mean().sort_values()
    paidOut=t[t.fromLeague.isin(leagueList)&(t.feeValue>0)].groupby('fromCountry')['ageFloat'].mean().sort_values()
    z=DataFrame({'all imports':allIn,'all exports':allOut,'free imports':freeIn,'free exports':freeOut,'paid imports':paidIn,'paid exports':paidOut}).sort_values('all imports')
    z=z[['all imports', 'all exports', 'paid imports', 'paid exports', 'free imports', 'free exports']]
    z.columns.name='average age'
    return z.reset_index().rename(columns={'index':'country'})

output=nonloans[(nonloans.fromCountry!=nonloans.toCountry)&(nonloans.season>2000)].groupby('period').apply(avgAge).unstack(level=0).swaplevel(axis=1).sort_index(axis=1,level=0)
output.to_excel('../data/analysis/average_age.xlsx')
output

period             (2000, 2005]                                       \
average age             country all imports all exports paid imports   
0                       Ireland   22.409149   23.529703    23.973114   
1                     Lithuania   23.829230   23.703737    20.531565   
2                   Netherlands   23.860759   26.440232    22.631212   
3                       Belgium   23.941876   25.433670    23.640985   
4                         Italy   24.095842   27.364584    24.095550   
5                       Ukraine   24.148060   25.964625    22.266673   
6                   Switzerland   24.224945   25.476489    24.863078   
7                       Finland   24.267914   26.184582    21.514473   
8            Bosnia-Herzegovina   24.382172   24.302822    23.195548   
9                    Kazakhstan   24.387747   26.002511    25.632285   
10               Czech Republic   24.447384   25.774304    23.630875   
11                       Serbia   24.514504   24.165868    22.276068   
12                      England   24.537455   27.843669    23.953204   
13                       France   24.566698   26.307493    23.308859   
14                       Russia   24.611994   26.930195    23.927922   
15                       Israel   24.626967   26.219730    24.441296   
16                   Luxembourg   24.720274   28.542516          NaN   
17                       Sweden   24.787623   25.967872    24.235953   
18                     Portugal   24.924613   27.571309    23.112042   
19                      Croatia   25.077151   25.980850    23.056828   
20                     Slovenia   25.161065   26.145761    19.892263   
21                        Spain   25.163496   28.838216    24.007526   
22                       Turkey   25.167983   27.337123    25.016186   
23                        Malta   25.192212   26.565182          NaN   
24                     Bulgaria   25.205987   25.944442    23.664415   
25                       Norway   25.250481   26.943487    25.747379   
26                     Scotland   25.263673   27.512516    25.100966   
27                      Denmark   25.284908   26.967039    25.336983   
28                      Germany   25.320799   29.129799    24.634123   
29                United States   25.411523   26.151092    23.788305   
30                   Azerbaijan   25.425437   24.327502    25.479647   
31                       Poland   25.554409   26.411950    23.381204   
32                       Greece   25.584299   27.598946    24.797881   
33                      Romania   25.852072   27.006314    25.225022   
34                     Slovakia   25.917202   25.579139    25.396825   
35                      Hungary   25.930146   27.370762    23.835827   
36                      Austria   26.390614   27.639546    25.199774   
37                       Cyprus   27.849420   28.504249    22.665421   

period                                                    (2005, 2010]  \
average age paid exports free imports free exports             country   
0              20.932212    22.034243    24.249414             Ireland   
1              20.992217    23.684887    23.924329               Italy   
2              25.085274    24.584446    25.599119             Belgium   
3              24.115485    23.573580    24.786286             England   
4              25.825943    24.927842    27.510049  Bosnia-Herzegovina   
5              24.455594    24.702988    26.234762              Russia   
6              23.980625    24.379445    25.600939             Finland   
7              22.338583    24.571632    25.180812            Portugal   
8              22.997392    24.360264    23.642983          Luxembourg   
9                    NaN    23.913918    23.435571              Sweden   
10             23.983697    25.135184    26.623009              Norway   
11             22.795219    24.913908    24.097415             Germany   
12             25.649321    25.646439    27.253574         Netherlands   
13       

In [5]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
from urllib2 import urlopen, Request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import numpy as np
import re
import os.path 
import datetime 
import math
import matplotlib.pyplot as plt

transfers = pd.read_csv('../data/merged/transfers.csv')
loans = pd.read_csv('../data/merged/loans.csv')
nonloans = pd.read_csv('../data/merged/nonloans.csv')
players=pd.read_csv('../data/merged/players.csv')

leagueCodeMapping = {'GB1':'Premier League (England)','FR1':'Ligue 1 (France)',
                                                         'ES1':'La Liga (Spain)','IT1':'Serie A (Italy)','L1':'Bundesliga (Germany)',
                                                         'PO1':'Primeira Liga (Portugal)','NL1':'Eredivisie (Netherlands)'}

teamIdMap = pd.concat([transfers[['toTeamId','toTeamName','season']].rename(columns={'toTeamId':'teamId','toTeamName':'teamName'}),
    transfers[['fromTeamId','fromTeamName','season']].rename(columns={'fromTeamId':'teamId','fromTeamName':'teamName'})],ignore_index=True) \
    .sort_values('season',ascending=False).drop_duplicates('teamId').set_index('teamId')['teamName'].to_dict()
    
agentIdMap = transfers[['playersAgentId','playersAgent']].sort_values('playersAgent',ascending=False).drop_duplicates('playersAgentId').set_index('playersAgentId')['playersAgent'].to_dict()

leagueList = ['GB1','ES1','L1','IT1','FR1','PO1','NL1','TR1','RU1','BE1','MLS1','UKR1','GR1','RO1','ZYP1','KR1','SER1','BOS1','BU1','DK1','SE1','NO1','LI1','SC1','ISR1','PL1','UNG1','A1','C1','MAL1','KAS1','TS1','SLO1','SL1','FI1','LUX1','AZ1','IR1']